In [55]:
from IPython.display import clear_output 
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import torch
from torchvision.utils import make_grid
from torchvision.io import read_image
# from torchvision import models, transforms
# import sys
# import os
from PIL import Image
import torch
# import torch.functional as F
import numpy as np
clear_output()
# import glob
from utils import *
from datetime import date
from torch.optim import lr_scheduler
import torchvision
from constants import mean, std
from PapilledemaLoader import PapilledemaDataset, PapilSeverityDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, recall_score

In [56]:
def run_test(pretrain_mode = 'ConPro', mode = 0, epo = 0):
    pretrain_mode = pretrain_mode

    data_dir = '/mnt/c/Users/PCM/Dropbox/chla_fundus_croped/severity'
    image_datasets = {x: PapilledemaDataset(data_dir=data_dir, phase=x) for x in ['test']}

    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4)
                for x in ['test']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['test']}
    class_names = image_datasets['test'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # print(device, class_names)

    bestmodel = models.resnet50(weights='ResNet50_Weights.DEFAULT')#get_feature_extractor(feature_extractor='resnet50', cotrain=False)#, simclr='/mnt/c/Users/PCM/Documents/GitHub/pseudopapill/SimCLR/runs/Oct29_21-00-13_DESKTOP-404G4HS/checkpoint_95_29102023.pth.tar')
    bestmodel.fc = nn.Sequential(torch.nn.Linear(2048, 1000),
                                    torch.nn.ReLU(),
                                    torch.nn.Dropout(0.1),
                                    torch.nn.Linear(1000, 256),
                                    torch.nn.ReLU(),
                                    torch.nn.Dropout(0.1),
                                    torch.nn.Linear(256, len(class_names)))
    bestmodel.load_state_dict(torch.load(f'./pretrained/{pretrain_mode}/best{mode}-{pretrain_mode}-{epo}.pt'))
    bestmodel.to(device)

    test_acc = 0
    predlist = []
    labelist = []
    problist = []
    sedis = 0

    for inputs, labels in dataloaders['test']:
        bestmodel.eval()
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = bestmodel(inputs)
            # emb = fextractor(inputs)
            _, preds = torch.max(outputs, 1)
            # loss = loss_fn(outputs, labels)
            sedis = sedis + torch.sum(torch.exp(torch.abs(labels - torch.max(outputs, 1)[1])))
        problist.append(outputs[:,1].detach().cpu().numpy())
        labelist.append(labels.detach().cpu().numpy()*1)
        predlist.append(preds.detach().cpu().numpy())
        # test_embeddings  = torch.cat((test_embeddings, emb.detach().cpu().flatten().unsqueeze(0)), axis=0)
        test_acc += torch.sum(preds == labels.data)

    labelist = np.concatenate(labelist).ravel()
    problist = np.concatenate(problist).ravel()
    predlist = np.concatenate(predlist).ravel()
    return sedis/dataset_sizes['test'], recall_score(labelist, predlist, average='macro'), f1_score(labelist, predlist, average='macro')
    # # print('MAEE', sedis/dataset_sizes['test'])
    # # print('weight F1', f1_score(labelist, predlist, average='weighted'))
    # # print('macro F1', f1_score(labelist, predlist, average='macro'))
    # # print(classification_report(labelist, predlist, digits=3))

In [69]:
maeelist = []
reclist = []
mf1list = []
pretrain_model = 'ConPro20'
for i in range(30):
    print('Run #',i)
    # maee, _, _ = run_test(pretrain_model, 'f1', i)
    # _, acc, _ = run_test(pretrain_model, 'f1', i)
    # _, _, mf1 = run_test(pretrain_model, 'f1', i)
    maee, rec, mf1 = run_test(pretrain_model, 'f1', i)
    # print('MAEE', maee)
    # print('Precision', acc)
    # print('macro F1', mf1)
    maeelist.append(maee)
    mf1list.append(mf1)
    reclist.append(rec)


Run # 0
Run # 1
Run # 2
Run # 3
Run # 4
Run # 5
Run # 6
Run # 7
Run # 8
Run # 9
Run # 10
Run # 11
Run # 12
Run # 13
Run # 14
Run # 15
Run # 16
Run # 17
Run # 18
Run # 19
Run # 20
Run # 21
Run # 22
Run # 23
Run # 24
Run # 25
Run # 26
Run # 27
Run # 28
Run # 29


In [70]:
print('mean and std of F1', np.mean(mf1list)*100, np.std(mf1list)*100)
print('mean and std of recall', np.mean(reclist)*100, np.std(reclist)*100)
print('mean and std of maee', np.mean(torch.stack(maeelist).cpu().numpy()), np.std(torch.stack(maeelist).cpu().numpy()))

mean and std of F1 46.257317733907826 5.090398139076438
mean and std of recall 47.525213675213685 4.301730684897399
mean and std of maee 5.0820975 0.2906615


In [24]:
maeelist

[tensor(5.2205, device='cuda:0'),
 tensor(4.7053, device='cuda:0'),
 tensor(4.7731, device='cuda:0'),
 tensor(5.4390, device='cuda:0'),
 tensor(4.8788, device='cuda:0'),
 tensor(4.5318, device='cuda:0'),
 tensor(4.7246, device='cuda:0'),
 tensor(4.8233, device='cuda:0'),
 tensor(4.5711, device='cuda:0'),
 tensor(4.6683, device='cuda:0'),
 tensor(4.6877, device='cuda:0'),
 tensor(5.1466, device='cuda:0'),
 tensor(4.7414, device='cuda:0'),
 tensor(4.4120, device='cuda:0'),
 tensor(5.0972, device='cuda:0'),
 tensor(4.9096, device='cuda:0'),
 tensor(4.4120, device='cuda:0'),
 tensor(4.6366, device='cuda:0'),
 tensor(5.0092, device='cuda:0'),
 tensor(5.2470, device='cuda:0'),
 tensor(4.6366, device='cuda:0'),
 tensor(4.7546, device='cuda:0'),
 tensor(4.8911, device='cuda:0'),
 tensor(4.3627, device='cuda:0'),
 tensor(5.0788, device='cuda:0'),
 tensor(4.9607, device='cuda:0'),
 tensor(4.5361, device='cuda:0'),
 tensor(5.1157, device='cuda:0'),
 tensor(4.8233, device='cuda:0'),
 tensor(4.7555